In [11]:
# preparation_data.py

import os
from io import BytesIO
import pandas as pd
from minio import Minio

# Configuration
file_path = r"E:\DOWNLOADS_G\Tự học\Taxi_Data_Insights\analyze_NYC_taxi_insight\data"
client = Minio(
    "localhost:9000",
    access_key="admin",
    secret_key="123456789",
    secure=False
)


# Step 1: Merge main parquet table
all_files = os.listdir(file_path)
dataframes = [pd.read_parquet(os.path.join(file_path, file)) for file in all_files]
merged_data_df = pd.concat(dataframes, ignore_index=True)
merged_data_df = merged_data_df.head(150000)
print("Merge successfully")


Merge successfully


In [12]:
merged_data_df.head(10)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2016-01-01 00:12:22,2016-01-01 00:29:14,1,3.20,1,N,48,262,1,14.0,0.5,0.5,3.06,0.0,0.3,18.36,None,None
1,1,2016-01-01 00:41:31,2016-01-01 00:55:10,2,1.00,1,N,162,48,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,None,None
2,1,2016-01-01 00:53:37,2016-01-01 00:59:57,1,0.90,1,N,246,90,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30,None,None
3,1,2016-01-01 00:13:28,2016-01-01 00:18:07,1,0.80,1,N,170,162,2,5.0,0.5,0.5,0.00,0.0,0.3,6.30,None,None
4,1,2016-01-01 00:33:04,2016-01-01 00:47:14,1,1.80,1,N,161,140,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30,None,None
5,1,2016-01-01 00:49:47,2016-01-01 01:04:44,1,2.30,1,N,141,137,1,11.0,0.5,0.5,2.45,0.0,0.3,14.75,None,None
6,1,2016-01-01 00:41:58,2016-01-01 01:22:06,1,13.80,1,N,100,53,2,43.0,0.5,0.5,0.00,0.0,0.3,44.30,None,None
7,2,2016-01-01 00:25:28,2016-01-01 00:55:46,5,3.46,1,N,48,79,2,20.0,0.5,0.5,0.00,0.0,0.3,21.30,None,None
8,2,2016-01-01 00:56:57,2016-01-01 01:02:24,4,0.83,1,N,79,107,2,5.5,0.5,0.5,0.00,0.0,0.3,6.80,None,None
9,2,2016-01-01 00:10:08,2016-01-01 00:23:05,1,0.87,1,N,164,164,2,7.0,0.5,0.5,0.00,0.0,0.3,8.30,None,None


In [13]:
# Step 2: Merge with taxi zone lookup
taxi_lookup_path = r"E:\DOWNLOADS_G\Tự học\Taxi_Data_Insights\analyze_NYC_taxi_insight\output_data\taxi_lookup.csv"
df_lookup = pd.read_csv(taxi_lookup_path)

# Merge for pickup locations
merged_data_df = merged_data_df.merge(df_lookup, left_on="PULocationID", right_on="LocationID")
merged_data_df = merged_data_df.drop(columns=['PULocationID', 'LocationID', 'Borough', 'service_zone', 'zone'])
merged_data_df = merged_data_df.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'})


In [14]:
merged_data_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,DOLocationID,payment_type,fare_amount,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Unnamed: 0,pickup_latitude,pickup_longitude
0,1,2016-01-01 00:12:22,2016-01-01 00:29:14,1,3.20,1,N,262,1,14.0,...,0.5,3.06,0.00,0.3,18.36,None,None,47,36.103413,-84.131863
1,1,2016-01-01 00:41:31,2016-01-01 00:55:10,2,1.00,1,N,48,2,9.5,...,0.5,0.00,0.00,0.3,10.80,None,None,161,35.821339,-78.620834
2,1,2016-01-01 00:53:37,2016-01-01 00:59:57,1,0.90,1,N,90,2,6.0,...,0.5,0.00,0.00,0.3,7.30,None,None,245,37.915937,-96.785860
3,1,2016-01-01 00:13:28,2016-01-01 00:18:07,1,0.80,1,N,162,2,5.0,...,0.5,0.00,0.00,0.3,6.30,None,None,169,38.290348,-85.588294
4,1,2016-01-01 00:33:04,2016-01-01 00:47:14,1,1.80,1,N,140,2,11.0,...,0.5,0.00,0.00,0.3,12.30,None,None,160,30.302121,-81.619652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2,2016-01-01 10:54:57,2016-01-01 11:04:00,2,1.77,1,N,186,1,8.5,...,0.5,2.79,0.00,0.3,12.09,None,None,160,30.302121,-81.619652
149996,1,2016-01-01 10:54:14,2016-01-01 11:20:33,1,13.90,1,N,161,2,39.0,...,0.5,0.00,5.54,0.3,45.34,None,None,137,40.775714,-73.873364
149997,2,2016-01-01 10:34:23,2016-01-01 10:41:24,2,1.20,1,N,236,1,7.0,...,0.5,1.56,0.00,0.3,9.36,None,None,261,33.465410,-88.358724
149998,2,2016-01-01 10:48:29,2016-01-01 10:52:51,1,0.36,1,N,75,2,4.5,...,0.5,0.00,0.00,0.3,5.30,None,None,235,-35.020944,138.652737


In [15]:
# Merge for dropoff locations
merged_data_df = merged_data_df.merge(df_lookup, left_on="DOLocationID", right_on="LocationID")
merged_data_df = merged_data_df.drop(columns=['DOLocationID', 'LocationID', 'Borough', 'service_zone', 'zone'])
merged_data_df = merged_data_df.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude'})

# Drop rows with missing values
merged_data_df = merged_data_df.drop(columns=['Unnamed: 0_x','Unnamed: 0_y']).dropna()

print(merged_data_df.head(10))

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2016-01-01 00:12:22   2016-01-01 00:29:14                1   
1         1  2016-01-01 00:41:31   2016-01-01 00:55:10                2   
2         1  2016-01-01 00:53:37   2016-01-01 00:59:57                1   
3         1  2016-01-01 00:13:28   2016-01-01 00:18:07                1   
4         1  2016-01-01 00:33:04   2016-01-01 00:47:14                1   
5         1  2016-01-01 00:49:47   2016-01-01 01:04:44                1   
6         1  2016-01-01 00:41:58   2016-01-01 01:22:06                1   
7         2  2016-01-01 00:25:28   2016-01-01 00:55:46                5   
8         2  2016-01-01 00:56:57   2016-01-01 01:02:24                4   
9         2  2016-01-01 00:10:08   2016-01-01 00:23:05                1   

   trip_distance  RatecodeID store_and_fwd_flag  payment_type  fare_amount  \
0           3.20           1                  N             1         14.0   
1           1.00  

In [16]:
merged_data_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,...,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Unnamed: 0_x,pickup_latitude,pickup_longitude,Unnamed: 0_y,dropoff_latitude,dropoff_longitude
0,1,2016-01-01 00:12:22,2016-01-01 00:29:14,1,3.20,1,N,1,14.0,0.5,...,0.3,18.36,None,None,47,36.103413,-84.131863,261,33.465410,-88.358724
1,1,2016-01-01 00:41:31,2016-01-01 00:55:10,2,1.00,1,N,2,9.5,0.5,...,0.3,10.80,None,None,161,35.821339,-78.620834,47,36.103413,-84.131863
2,1,2016-01-01 00:53:37,2016-01-01 00:59:57,1,0.90,1,N,2,6.0,0.5,...,0.3,7.30,None,None,245,37.915937,-96.785860,89,44.326509,-103.768963
3,1,2016-01-01 00:13:28,2016-01-01 00:18:07,1,0.80,1,N,2,5.0,0.5,...,0.3,6.30,None,None,169,38.290348,-85.588294,161,35.821339,-78.620834
4,1,2016-01-01 00:33:04,2016-01-01 00:47:14,1,1.80,1,N,2,11.0,0.5,...,0.3,12.30,None,None,160,30.302121,-81.619652,139,40.773643,-73.960862
5,1,2016-01-01 00:49:47,2016-01-01 01:04:44,1,2.30,1,N,1,11.0,0.5,...,0.3,14.75,None,None,140,37.742295,-122.465106,136,40.739546,-73.977083
6,1,2016-01-01 00:41:58,2016-01-01 01:22:06,1,13.80,1,N,2,43.0,0.5,...,0.3,44.30,None,None,99,40.753694,-73.990517,52,40.787601,-73.845968
7,2,2016-01-01 00:25:28,2016-01-01 00:55:46,5,3.46,1,N,2,20.0,0.5,...,0.3,21.30,None,None,47,36.103413,-84.131863,78,40.729269,-73.987361
8,2,2016-01-01 00:56:57,2016-01-01 01:02:24,4,0.83,1,N,2,5.5,0.5,...,0.3,6.80,None,None,78,40.729269,-73.987361,106,40.735519,-73.984079
9,2,2016-01-01 00:10:08,2016-01-01 00:23:05,1,0.87,1,N,2,7.0,0.5,...,0.3,8.30,None,None,163,40.749842,-73.984251,163,40.749842,-73.984251


In [17]:
tinmom = merged_data_df

In [18]:
merged_data_df = merged_data_df.drop(columns=['Unnamed: 0_x','Unnamed: 0_y']).d


In [19]:
merged_data_df.head(10)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,1,2016-01-01 00:12:22,2016-01-01 00:29:14,1,3.20,1,N,1,14.0,0.5,...,3.06,0.0,0.3,18.36,None,None,36.103413,-84.131863,33.465410,-88.358724
1,1,2016-01-01 00:41:31,2016-01-01 00:55:10,2,1.00,1,N,2,9.5,0.5,...,0.00,0.0,0.3,10.80,None,None,35.821339,-78.620834,36.103413,-84.131863
2,1,2016-01-01 00:53:37,2016-01-01 00:59:57,1,0.90,1,N,2,6.0,0.5,...,0.00,0.0,0.3,7.30,None,None,37.915937,-96.785860,44.326509,-103.768963
3,1,2016-01-01 00:13:28,2016-01-01 00:18:07,1,0.80,1,N,2,5.0,0.5,...,0.00,0.0,0.3,6.30,None,None,38.290348,-85.588294,35.821339,-78.620834
4,1,2016-01-01 00:33:04,2016-01-01 00:47:14,1,1.80,1,N,2,11.0,0.5,...,0.00,0.0,0.3,12.30,None,None,30.302121,-81.619652,40.773643,-73.960862
5,1,2016-01-01 00:49:47,2016-01-01 01:04:44,1,2.30,1,N,1,11.0,0.5,...,2.45,0.0,0.3,14.75,None,None,37.742295,-122.465106,40.739546,-73.977083
6,1,2016-01-01 00:41:58,2016-01-01 01:22:06,1,13.80,1,N,2,43.0,0.5,...,0.00,0.0,0.3,44.30,None,None,40.753694,-73.990517,40.787601,-73.845968
7,2,2016-01-01 00:25:28,2016-01-01 00:55:46,5,3.46,1,N,2,20.0,0.5,...,0.00,0.0,0.3,21.30,None,None,36.103413,-84.131863,40.729269,-73.987361
8,2,2016-01-01 00:56:57,2016-01-01 01:02:24,4,0.83,1,N,2,5.5,0.5,...,0.00,0.0,0.3,6.80,None,None,40.729269,-73.987361,40.735519,-73.984079
9,2,2016-01-01 00:10:08,2016-01-01 00:23:05,1,0.87,1,N,2,7.0,0.5,...,0.00,0.0,0.3,8.30,None,None,40.749842,-73.984251,40.749842,-73.984251


In [20]:
merged_data_df = merged_data_df.dropna()


In [21]:
merged_data_df.head(10)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude


In [23]:
merged_data_df = tinmom


In [28]:

# Drop rows with missing values
merged_data_df.drop(columns=['Unnamed: 0_x','Unnamed: 0_y']).dropna(how='all')



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,1,2016-01-01 00:12:22,2016-01-01 00:29:14,1,3.20,1,N,1,14.0,0.5,...,3.06,0.00,0.3,18.36,None,None,36.103413,-84.131863,33.465410,-88.358724
1,1,2016-01-01 00:41:31,2016-01-01 00:55:10,2,1.00,1,N,2,9.5,0.5,...,0.00,0.00,0.3,10.80,None,None,35.821339,-78.620834,36.103413,-84.131863
2,1,2016-01-01 00:53:37,2016-01-01 00:59:57,1,0.90,1,N,2,6.0,0.5,...,0.00,0.00,0.3,7.30,None,None,37.915937,-96.785860,44.326509,-103.768963
3,1,2016-01-01 00:13:28,2016-01-01 00:18:07,1,0.80,1,N,2,5.0,0.5,...,0.00,0.00,0.3,6.30,None,None,38.290348,-85.588294,35.821339,-78.620834
4,1,2016-01-01 00:33:04,2016-01-01 00:47:14,1,1.80,1,N,2,11.0,0.5,...,0.00,0.00,0.3,12.30,None,None,30.302121,-81.619652,40.773643,-73.960862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2,2016-01-01 10:54:57,2016-01-01 11:04:00,2,1.77,1,N,1,8.5,0.0,...,2.79,0.00,0.3,12.09,None,None,30.302121,-81.619652,40.750347,-73.991203
149996,1,2016-01-01 10:54:14,2016-01-01 11:20:33,1,13.90,1,N,2,39.0,0.0,...,0.00,5.54,0.3,45.34,None,None,40.775714,-73.873364,30.302121,-81.619652
149997,2,2016-01-01 10:34:23,2016-01-01 10:41:24,2,1.20,1,N,1,7.0,0.0,...,1.56,0.00,0.3,9.36,None,None,33.465410,-88.358724,-35.020944,138.652737
149998,2,2016-01-01 10:48:29,2016-01-01 10:52:51,1,0.36,1,N,2,4.5,0.0,...,0.00,0.00,0.3,5.30,None,None,-35.020944,138.652737,NaN,NaN
